In [152]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,recall_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Dropout,Embedding,Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import BinaryCrossentropy

import re
from tensorflow.keras.preprocessing.text import Tokenizer

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize



nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [153]:
df = pd.read_csv('file3.csv')

In [154]:
df.info

<bound method DataFrame.info of                    author                                          statement  \
0      Samantha Putterman  Joe Biden "put 20 million people on Medicaid w...   
1             Grace Abels  An ABC News headline said, “Journalist coverin...   
2              Sara Swann  "Bill Gates mRNA 'air vaccine' approved for us...   
3        Aubrey Burkhardt  “The percentage of students that are going to ...   
4      Michelle Peribanez  “Gas prices have skyrocketed since Joe Biden t...   
...                   ...                                                ...   
13465           John Hill  In Rhode Island, "Nearly 9 percent of covered ...   
13466   Nancy Badertscher  "Georgia has the nation's fifth largest Women,...   
13467        Jim Haddadin  "Shaheen votes with Obama 99 percent of the ti...   
13468        Nancy Madsen  "From 1972 until 2011, the number of EPA emplo...   
13469         Amy Sherman  "My state of Florida is now the third-largest ...   

       

In [155]:
df.head()

,author,statement,source,date,target
0,Samantha Putterman,"Joe Biden ""put 20 million people on Medicaid w...",Nikki Haley,"October 6, 2023",FALSE
1,Grace Abels,"An ABC News headline said, “Journalist coverin...",TikTok posts,"October 6, 2023",FALSE
2,Sara Swann,"""Bill Gates mRNA 'air vaccine' approved for us...",Instagram posts,"October 6, 2023",FALSE
3,Aubrey Burkhardt,“The percentage of students that are going to ...,George Zimmerman,"October 6, 2023",mostly-true
4,Michelle Peribanez,“Gas prices have skyrocketed since Joe Biden t...,Carol Miller,"October 6, 2023",half-true


In [156]:
df['source'].value_counts()

source
Facebook posts                         1962
Viral image                             959
Donald Trump                            849
Bloggers                                647
Instagram posts                         632
                                       ... 
Fellowship of the Minds                   1
The Trump Talks                           1
Jose Carrillo                             1
Viral Truth Wire                          1
Georgia Department of Public Health       1
Name: count, Length: 2808, dtype: int64

In [157]:
df['target'].unique()

array(['FALSE', 'mostly-true', 'half-true', 'pants-fire', 'barely-true',
       'TRUE', 'full-flop', 'half-flip', 'no-flip'], dtype=object)

In [158]:
category_mapping={
    'FALSE':0.0,
    'pants-fire':0.0,
    'full-flop':0.1,
    'barely-true':0.2,
    'half-flip':0.4,
    'half-true':0.5,
    'no-flip':0.6,
    'mostly-true':0.9,
    'TRUE':1.0
}

In [159]:
df['target']=df['target'].map(category_mapping)
# df['target']=df['target'].values


In [160]:
type(df['target'].values)

numpy.ndarray

In [161]:
x=df['statement']

In [162]:
port_stem = PorterStemmer()


In [163]:
def stemming(Content):
    stemmed_con = re.sub('[^A-Za-z]',' ',Content)
    stemmed_con = stemmed_con.lower()
    stemmed_con = stemmed_con.split()
    stemmed_con = [port_stem.stem(word) for word in stemmed_con if not word in stopwords.words('english')]
    stemmed_con = ' '.join(stemmed_con)
    return stemmed_con

In [164]:
x=x.apply(stemming)

In [165]:
X_train, X_test, y_train, y_test = train_test_split(x,df['target'] , test_size=0.20, random_state=18)

In [166]:
y_test

7361     0.0
7387     0.0
6504     0.0
2512     0.5
2696     0.0
        ... 
8589     0.0
854      0.0
10783    0.5
9364     0.9
6130     0.9
Name: target, Length: 2694, dtype: float64

In [167]:
max_vocab=70
tokenizer=Tokenizer(70)

tokenizer.fit_on_texts(X_train)
tokenizer.fit_on_texts(X_test)


In [168]:
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [169]:
k=[]
for i in x:
    print(len(i))
    k.append(len(i))
sum(k)/len(k)

62
70
54
58
61
36
78
91
76
116
46
54
51
26
49
109
40
32
24
41
50
38
72
72
33
42
43
33
123
66
33
24
29
85
56
47
83
58
45
56
43
151
34
53
60
53
44
27
38
30
71
58
81
48
28
87
84
23
146
20
37
58
62
103
32
110
59
106
46
83
71
44
61
58
163
120
24
55
70
71
92
51
182
73
27
27
67
38
25
97
46
34
43
41
54
45
55
35
37
86
38
38
55
51
99
65
89
62
41
42
85
55
50
60
62
77
59
18
37
41
38
55
56
55
42
32
30
79
35
51
59
36
73
62
44
49
91
53
47
55
51
47
41
65
27
65
103
52
67
44
91
46
30
55
135
131
63
98
59
66
76
66
69
63
61
51
57
125
76
42
97
163
44
43
76
21
96
98
50
50
91
16
46
15
39
59
81
22
56
38
46
21
172
29
43
66
104
76
47
61
64
38
75
67
35
67
68
38
53
81
53
110
39
59
87
50
48
34
72
48
34
32
54
82
40
39
81
82
56
26
51
29
35
95
34
41
108
63
55
80
77
39
73
59
71
39
62
67
40
61
45
67
94
31
63
47
53
87
21
98
71
41
46
93
42
70
18
95
58
26
91
53
101
72
95
40
66
71
48
90
97
72
73
69
87
80
41
85
49
43
153
96
62
70
54
139
48
14
60
76
39
59
36
106
147
50
62
64
60
91
57
61
70
55
145
107
63
56
90
53
35
39
33
62
2

63.71930215293244

In [170]:
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, padding='post', maxlen=256)
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, padding='post', maxlen=256)

In [171]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(max_vocab, 128),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)),
    tf.keras.layers.Dense(64, activation='relu'),
    # tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

In [172]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, None, 128)         8960      
                                                                 
 bidirectional_12 (Bidirect  (None, None, 128)         98816     
 ional)                                                          
                                                                 
 bidirectional_13 (Bidirect  (None, 32)                18560     
 ional)                                                          
                                                                 
 dense_12 (Dense)            (None, 64)                2112      
                                                                 
 dense_13 (Dense)            (None, 1)                 65        
                                                                 
Total params: 128513 (502.00 KB)
Trainable params: 128

In [173]:
# early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=20,validation_split=0.1, batch_size=30, shuffle=True)

Epoch 1/20
324/324 [==============================] - 140s 385ms/step - loss: 0.6131 - accuracy: 0.4955 - val_loss: 0.5959 - val_accuracy: 0.4954
Epoch 2/20
 26/324 [=>............................] - ETA: 1:23 - loss: 0.6010 - accuracy: 0.4833

KeyboardInterrupt: 

In [ ]:
stop_words = set(stopwords.words('english')) 
def LemmSentence(sentence):
    lemma_words = []
    wordnet_lemmatizer = WordNetLemmatizer()
    word_tokens = word_tokenize(sentence) 
    for word in word_tokens: 
        if word not in stop_words: 
            new_word = re.sub('[^a-zA-Z]', '',word)
            new_word = new_word.lower()
            new_word = wordnet_lemmatizer.lemmatize(new_word)
            lemma_words.append(new_word)
    return " ".join(lemma_words)

X = [LemmSentence(i) for i in x]

LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/wordnet[0m

  Searched in:
    - 'C:\\Users\\hp/nltk_data'
    - 'c:\\Program Files\\Python311\\nltk_data'
    - 'c:\\Program Files\\Python311\\share\\nltk_data'
    - 'c:\\Program Files\\Python311\\lib\\nltk_data'
    - 'C:\\Users\\hp\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [ ]:
import re
import string
def wp(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [ ]:
df["text"] = df["text"].apply(wp)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 42)